# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Начнем нашу работу с импорта всех необходимых библиотек. По сути, перед нами стоит задача классификации между двумя известными тарифами. Следовательно. нам необходимо импортировать различные Classifier-библиотеки. Так же по заданию нам необходимо ориентироваться на метрику accuracy, следоватьелньо, нам необходим модуль для вычисления данной метрики.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

## Откройте и изучите файл

Открываем и изучаем данные:

In [2]:
df=pd.read_csv('D:\\DS\\Портфолио\\5-Машинное обучение\\users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Отлично, у нас нет пропусков и все данные необходимого формата.

In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Необычных данных тоже нет.

**Вывод:** 
Отлично, мы можем сразу приступать к разделению данных на выборки.

## Разбейте данные на выборки

Целевым столбцом является для нас столбец "is_ultra". Остальные столбцы являются признаками. Создадим переменные:

In [5]:
features=df.drop(['is_ultra'], axis=1)
target=df['is_ultra']

#првоерим:
features.head()

,calls,minutes,messages,mb_used
0,40.0,311.90,83.0,19915.42
1,85.0,516.75,56.0,22696.96
2,77.0,467.66,86.0,21060.45
3,106.0,745.53,81.0,8437.39
4,66.0,418.74,1.0,14502.75


In [6]:
#провервим:
target.head()

0    0
1    0
2    0
3    1
4    0
Name: is_ultra, dtype: int64

Для выполнения работы нам понадобится разделить данные на 3 пары выборок, примерно в следующих пропорциях:

-Обучающую (60% выборок)

-Валидационную (20% выборок)

-Тестовую (20% выборок)

Сперва мы разделим выборку на две части: df1 и test. 
test мы будем использовать для финальной проверки, а df1 разделим еще на 2 выборки: train_df и valid_df.

Во всем проекте мы будем использовать единый random_state=1123 (со второго по пятое число последовательности Фибоначчи)

In [7]:
df1_features, test_features_df, df1_target, test_target_df=train_test_split(
features, target, test_size=0.20, random_state=1123)

Отлично теперь разделим df1 выборку на обучающую и валидационную:

In [8]:
train_features_df, valid_features_df, train_target_df, valid_target_df=train_test_split(
df1_features, df1_target, test_size=0.20, random_state=1123)

Проверим размерность наших наборов:

In [9]:
train_features_df.shape

(2056, 4)

In [10]:
train_target_df.shape

(2056,)

In [11]:
valid_features_df.shape

(515, 4)

In [12]:
valid_target_df.shape

(515,)

In [13]:
test_features_df.shape

(643, 4)

In [14]:
test_target_df.shape

(643,)

**Вывод:**

Отлично, итого у нас получилось 3 пары фреймов:

train_features_df и train_target_df - для обучения

valid_features_df и valid_target_df - для валидации

test_features_df и test_target_df - для финального тестирования.


## Исследуйте модели

Приступаем к созданию, обучению и валидации моделей. Главу разобьем на 4 подглавы:

1) DecisionTree

2) RandomForest

3) LogisticRegression

4) Выбор наилучшей модели

На протяжении всего исследования мы будем использовать повторяющиеся команды по обучению и вычислению результата. Поэтому запишем их в функцию:

In [15]:
def fit_result(train_feature_df, train_target_df, valid_feature_df, valid_target_df):
    
    '''
    Вычисляет accuracy по по валидационной выборке. 
    
    Внутри зашито обучение модели, предсказание на валидационной выборке и вычисление accuracy
    ======================================
    
    Аргументы:
                    train_feature_df (DataFrame): датафрейм признаков из тренировочной выборки
                    train_target_df (DataFrame): датафрейм целевого признака из тренировочной выборки
                    valid_feature_df (DataFrame): датафрейм признаков из валидационной выборки
                    train_target_df (DataFrame): датафрейм целевого признака из валидационной  выборки
                    

    Возвращает:
                    result (float): accuracy вычисленное на валидационной выборке
    '''    
    model.fit(train_feature_df,train_target_df)
    prediction_valid=model.predict(valid_feature_df)
    result = accuracy_score(valid_target_df, prediction_valid)
    return result

print('Строка документации: \n', fit_result.__doc__)

Строка документации: 
 
    Вычисляет accuracy по по валидационной выборке. 
    
    Внутри зашито обучение модели, предсказание на валидационной выборке и вычисление accuracy
    
    Аргументы:
                    train_feature_df (DataFrame): датафрейм признаков из тренировочной выборки
                    train_target_df (DataFrame): датафрейм целевого признака из тренировочной выборки
                    valid_feature_df (DataFrame): датафрейм признаков из валидационной выборки
                    train_target_df (DataFrame): датафрейм целевого признака из валидационной  выборки
                    

    Возвращает:
                    result (float): accuracy вычисленное на валидационной выборке
    


### DecisionTree

В данной главе мы будем работать с алгоритмом DecisionTree.

Мы будем регулировать 3 гипер параметра в данной модели: 

-max_depth 

-min_samples_split

-min_samples_leaf



In [16]:
best_tree=None
best_accuracy=0
best_depth=0
best_split=0
best_leaf=0

for depth in range(1,10):
    for split in range(2,5):
        for leaf in range(1,4):
            model=DecisionTreeClassifier(random_state=1123, max_depth=depth, min_samples_split=split, min_samples_leaf=leaf) 
            if fit_result(train_features_df, train_target_df, valid_features_df, valid_target_df)>best_accuracy:
                best_tree=model
                best_accuracy=fit_result(train_features_df, train_target_df, valid_features_df, valid_target_df)
                best_depth=depth
                best_split=split
                best_leaf=leaf
                
print('accuracy наилучшей модели на валидационной выборке: {}'.format(best_accuracy))
print('depth наилучшей модели на валидационной выборке: {}'.format(best_depth))
print('split наилучшей модели на валидационной выборке: {}'.format(best_split))
print('leaf наилучшей модели на валидационной выборке: {}'.format(best_leaf))

accuracy наилучшей модели на валидационной выборке: 0.8155339805825242
depth наилучшей модели на валидационной выборке: 7
split наилучшей модели на валидационной выборке: 4
leaf наилучшей модели на валидационной выборке: 1


Отлично, мы получили модель с необходимым accuracy, а так же наилучшие гиперпараметры модели. Давайте посмотрим на время предсказания на валидационной выборке:

In [17]:
%%time 

prediction_valid=best_tree.predict(valid_features_df)

Wall time: 7 ms


**Вывод:**

Наилучшая модель работающая на алгоритме DecisionTree имеет следующие точность и гиперпараметры:

accuracy наилучшей модели на валидационной выборке: 0.8155339805825242
depth наилучшей модели на валидационной выборке: 7
split наилучшей модели на валидационной выборке: 4
leaf наилучшей модели на валидационной выборке: 1

И выполняет предсказание за 1,68 миллисекунд на валидационной выборке. Наверное это быстро.

### RandomForest

В данной главе мы будем работать с алгоритмом RandomForest.

Мы будем регулировать 2 гипер параметра в данной модели:

-max_depth 

-n_estimator 

In [18]:
best_forest=None
best_forest_accuracy=0
best_forest_depth=0
best_forest_est=0

for depth in range(1,10):
    for est in range(10,51,10):
        model=RandomForestClassifier(random_state=1123, max_depth=depth, n_estimators=est) 
        if fit_result(train_features_df, train_target_df, valid_features_df, valid_target_df)>best_forest_accuracy:
            best_forest=model
            best_forest_accuracy=fit_result(train_features_df, train_target_df, valid_features_df, valid_target_df)
            best_forest_depth=depth
            best_forest_est=est
           
                
print('accuracy наилучшей модели на валидационной выборке: {}'.format(best_forest_accuracy))
print('depth наилучшей модели на валидационной выборке: {}'.format(best_forest_depth))
print('est наилучшей модели на валидационной выборке: {}'.format(best_forest_est))

accuracy наилучшей модели на валидационной выборке: 0.8368932038834952
depth наилучшей модели на валидационной выборке: 9
est наилучшей модели на валидационной выборке: 30


Отлично, мы получили модель с необзодимым accuracy, а так же наилучшие гиперпараметры модели. Давайте посмотрим на время выполнения предсказания на валидационной выборке:

In [19]:
%%time 

prediction_valid=best_forest.predict(valid_features_df)

Wall time: 35 ms


**Вывод:**

Наилучшая модель работающая на алгоритме RandomForest имеет следующие точность и гиперпараметры:

accuracy наилучшей модели на валидационной выборке: 0.8368932038834952
depth наилучшей модели на валидационной выборке: 9
est наилучшей модели на валидационной выборке: 30

И выполняет предсказание за 9,48 миллисекунд на валидационной выборке. Это медленнее чем DecisionTree. 
Причем accuracy больше на совсем небольшое значение, а вот скорость работы в несколько раз медленнее.

### LogisticRegression

В данной главе мы будем работать с алгоритмом LogisticRegression.

In [20]:
best_log_regr=None
best_log_regr_accuracy=0
best_log_regr_c=0


for c in range(1,10):
    model=LogisticRegression(random_state=1123, C=c)
    if fit_result(train_features_df, train_target_df, valid_features_df, valid_target_df)>best_log_regr_accuracy:
        best_log_regr=model
        best_log_regr_accuracy=fit_result(train_features_df, train_target_df, valid_features_df, valid_target_df)
        best_log_regr_c=c
                        
print('accuracy наилучшей модели на валидационной выборке: {}'.format(best_log_regr_accuracy))
print('C наилучшей модели на валидационной выборке: {}'.format(best_log_regr_c))

accuracy наилучшей модели на валидационной выборке: 0.7300970873786408
C наилучшей модели на валидационной выборке: 1


Мы получили accuracy ниже требуемого. Нет смысла дальше использовать данный алгоритм.

### Выбор лучшего алгоритма

Исходя из accuracy и скоростью работы алгоритмов, для дальнейшей работы лучше использовать DecisionTree. Проверим ее на тестовой выборке.

## Проверьте модель на тестовой выборке

Проверим работу нашей модели на тестовой выборке.

In [21]:
#Делаем предсказание на тестовой выборке
prediction_test=best_tree.predict(test_features_df)

#Вычисляем accuracy на тестовой выборке
accu_test=accuracy_score(test_target_df, prediction_test)

print('accuracy на тестовой выборке: {}'.format(accu_test))

accuracy на тестовой выборке: 0.8055987558320373


**Вывод:**

Мы обучили модель используя алгоритм DecisionTree. Нам удалось получить accuracy=0.80 выше требуемого (0,75).
Мы отсеяли алгоритм RandomForest из-за медлительности, а LogisticRegression из-за низкого accuracy.

## (бонус) Проверьте модели на адекватность

Предположим, перед нами модель, которая предсказывает тарифы случайным образом. С вероятностью 50/50 ответ будет 0 или 1. 
Тогда accuracy такой модели следующая:

accuracy=0.5x(доля угаданных 1) + 0.5х(доля угаданных 0)

Ответы модели не связаны с правильными ответами, поэтому вероятность угадать 1 равна 50% (так же и для 0)

тогда accuracy = 0.5x0.5 + 0.5x0.5=0.25+0.25 = 0.5

Выполним проверку при промощи DummyClassifier

In [22]:
dummy_clf = DummyClassifier(random_state=1123, strategy="most_frequent")
dummy_clf.fit(train_features_df,train_target_df)
prediction_dummy_test=dummy_clf.predict(test_features_df)
accu_dummy_test=accuracy_score(test_target_df, prediction_dummy_test)

print('accuracy на тестовой выборке: {}'.format(accu_dummy_test))

accuracy на тестовой выборке: 0.71850699844479


**Вывод:**

Наша модель имеет accuracy (0.80) выше чем accuracy случайной модели (0.5), и выше, чем модель реализованная через DummyClassifier (0.71). Следовательно наша модель прошла проверку на адекватность.

## Вывод

Мы обучили несколько моделей и использовали в работе различные алгоритмы, а именно:

DecisionTree

RandomForest

LogisticRegression

Нам не подошел алгортим LogisticRegression, так как accuracy на модели ниже требуемого. (0.72/0.75)

Модели работающих на адлгоритмах DecisionTree и RandomForest показали accuracy выше необходимого (0.80/0.75 на тестовой выборке) и (0.83/0.75 на валидационной выборке).

Алгоритм RandomForest оказался в несколько раз медленне алгоритма DecisionTree, но в то же время оказался несколько точнее.

Можно предложить заказчику на рассмотрение оба варианта, на случай если высокая точность будет важнее производительности.

Дополнительно мы провели проверку на адекватность при помощи DummyClassifier и случайной моделью. Отобранные нами модели показывают лучший результат и прошли проверку на адекватность.